In [14]:
from sklearn.metrics import ndcg_score
from src.Common import get_pickle
import numpy as np

preds = get_pickle("", "pred")
labels = get_pickle("", "labels")

In [15]:
len(preds)

3935

In [23]:
y_pred = preds[case]
y_true = np.zeros(len(y_pred))
y_true[labels[case]] = 1

ndcg_score([y_true], [y_pred])

0.2702381544273198

In [28]:
all_ndcg = []
for case in range(len(preds)):
    y_pred = preds[case]
    y_true = np.zeros(len(y_pred))
    y_true[labels[case]] = 1
    ndcg = ndcg_score([y_true], [y_pred], k=5)
    all_ndcg.append(ndcg)

In [29]:
np.mean(all_ndcg)

0.5525731790955378